In [2]:
from Environment import GridworldEnv
import numpy as np

In [12]:
a = np.arange(24).reshape(2,3,4)
print("origin:",a.shape) # (3, 4)
print("no axis:",np.max(a).shape) # (1)
print("axis=0:",np.max(a, axis=0).shape) # (4)
print("axis=1:",np.max(a, axis=1).shape) # (3)
print("axis=2:",np.max(a, axis=2).shape) # (3)

origin: (2, 3, 4)
no axis: ()
axis=0: (3, 4)
axis=1: (2, 4)
axis=2: (2, 3)


In [47]:
def cal_action_values(environment, state, state_values, discount= 1.0):
    "对某一状态而言的动作价值函数"
    env = environment
    "初始化动作价值函数"
    action_values = np.zeros((env.nA), dtype=np.float32)
    for action in range(env.nA):
        for prob, next_state, reward, done in env.P[state][action]:
            action_values[action] += prob * (reward + discount * state_values[next_state])
    
    return action_values # (env.nA)

In [48]:
def value_iteration(environment, theta = 1e-1, discount = 1.0):
    env = environment
    "初始化状态价值函数"
    state_values = np.zeros((env.nS), dtype=np.float32)
    max_eposides = 50
    
    "值迭代核心算法"
    for eposide in range(max_eposides):
        delta = 0.
        "对每一状态迭代"
        for state in range(env.nS):
            action_values = cal_action_values(env, state, state_values, discount) # (nA)
            optim_action_values = np.max(action_values)
            delta = np.maximum(delta, np.abs(optim_action_values - state_values[state]))
            state_values[state] = optim_action_values

        # print("eposide:", eposide, "state_values",state_values, "\n")
        if(delta <= theta):
            break
    
    "策略评估"
    optim_policy = np.zeros((env.nS, env.nA), dtype=np.float32)
    for state in range(env.nS):
        action_values = cal_action_values(env, state, state_values, discount) # (nA)
        optim_action = np.argmax(action_values)
        optim_policy[state, optim_action] = 1.
    return optim_policy, state_values

In [49]:
env = GridworldEnv()
policy, state_values = value_iteration(env) # (nS, nA), (nS)
print(np.argmax(policy, axis=1).reshape(env.desc.shape))
print(state_values.reshape(env.desc.shape))

[[1 1 2 3]
 [2 0 2 0]
 [1 1 2 0]
 [0 1 1 0]]
[[ -6.  -5.  -4.  -5.]
 [ -5. -50.  -3. -50.]
 [ -4.  -3.  -2. -50.]
 [-50.  -2.  -1.   0.]]
